<a href="https://colab.research.google.com/github/Phuocdatng24/Python/blob/main/Check_th%E1%BB%A9_h%E1%BA%A1ng_t%E1%BB%AB_kh%C3%B3a_%7C_ver_1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ✅ Tool Check thứ hạng từ khóa Google 1️⃣2️⃣3️⃣ { form-width: "100%", display-mode: "form" }
# @markdown 👈Nhấn nút này để chạy tool thì form dưới mới nhập được.<br>
# @markdown <font color="red">**File sẽ tải tự động sau khi chạy xong.**</font><br>
# @markdown **Ver:** <font color="red">**1.0.1**</font> | **Update:** <font color="blue">08/05/2024</font>
import pandas as pd
import json
import requests
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# Hàm thực hiện gửi yêu cầu và lưu kết quả vào cột mới
def save_result_to_excel(api_key, df_keywords, file_name, domain_check, quoc_gia, ngon_ngu):
    total_keywords = len(df_keywords)
    progress_bar = widgets.IntProgress(min=0, max=total_keywords, description='Tiến trình:')
    display(progress_bar)

    for index, row in df_keywords.iterrows():
        keyword = row.iloc[0]  # Lấy từ khóa từ cột đầu tiên
        if keyword == '':
            continue

        # Tạo payload và gửi yêu cầu tới API
        payload = json.dumps({
            "q": keyword,
            "gl": quoc_gia,
            "hl": ngon_ngu,
            "num": 100
        })
        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }

        # Gửi yêu cầu tới API và lưu dữ liệu JSON vào cột mới
        response = requests.request("POST", "https://google.serper.dev/search", headers=headers, data=payload)
        json_data = response.json()

        # Kiểm tra domain_check trong kết quả JSON
        domain_position = 100  # Mặc định nếu không tìm thấy domain
        link = ""  # Mặc định nếu không tìm thấy link
        if 'organic' in json_data:
            for item in json_data['organic']:
                if 'link' in item and domain_check in item['link']:
                    domain_position = item.get('position', 100)
                    link = item['link']
                    break

        df_keywords.at[index, 'domain_position'] = domain_position
        df_keywords.at[index, 'link'] = link

        # Cập nhật tiến trình
        progress_bar.value += 1

    # Đổi tên cột domain_position thành "Thứ hạng"
    df_keywords.rename(columns={'domain_position': 'Thứ hạng'}, inplace=True)

    # Lưu dữ liệu vào file Excel và cho phép tải file xuống máy tính
    df_keywords.to_excel(file_name, index=False)
    files.download(file_name)

# --- Tạo các widget (hộp nhập liệu và nút bấm) ---

# Tạo hộp nhập liệu cho API Key
api_key_input = widgets.Text(
    description='API Key:',
    placeholder='Nhập API Key',
    disabled=False
)

# Tạo hộp nhập liệu cho Domain Check
domain_check_input = widgets.Text(
    description='Domain:',
    placeholder='VD: dienmayxanh.com',
    disabled=False
)

# Tạo hộp chọn cho Quốc gia
quoc_gia_options = ["vn", "us", "jp", "au"]
quoc_gia_dropdown = widgets.Dropdown(
    options=quoc_gia_options,
    value='vn',
    description='Quốc gia:',
    disabled=False,
)

# Tạo hộp chọn cho Ngôn ngữ
ngon_ngu_options = ["vi", "en", "ja", "au"]
ngon_ngu_dropdown = widgets.Dropdown(
    options=ngon_ngu_options,
    value='vi',
    description='Ngôn ngữ:',
    disabled=False,
)

# Tạo hộp nhập liệu cho danh sách từ khóa (Textarea)
text_area = widgets.Textarea(
    value='',
    placeholder='Nhập từ khóa, mỗi từ khóa một dòng',
    description='Từ khóa:',
    disabled=False,
    layout=widgets.Layout(width='500px', height='200px')
)

# Tạo nút bấm để xử lý
button = widgets.Button(description="Chạy")

# --- Hàm xử lý khi bấm nút ---

def on_button_click(b):
    # Lấy giá trị từ các hộp nhập liệu
    api_key = api_key_input.value
    domain_check = domain_check_input.value
    quoc_gia = quoc_gia_dropdown.value
    ngon_ngu = ngon_ngu_dropdown.value
    keywords = text_area.value.strip().splitlines()

    # Tạo DataFrame từ danh sách từ khóa
    df_keywords = pd.DataFrame({'Từ khóa': keywords})
    df_keywords['domain_position'] = ""  # Thêm cột để lưu trữ position

    # Gọi hàm để gửi yêu cầu và lưu kết quả
    file_name = "ket_qua_kiem_tra.xlsx"
    save_result_to_excel(api_key, df_keywords, file_name, domain_check, quoc_gia, ngon_ngu)

# Gắn hàm xử lý vào nút bấm
button.on_click(on_button_click)

# --- Hiển thị các widget trong cell riêng ---

display(api_key_input, domain_check_input, quoc_gia_dropdown, ngon_ngu_dropdown, text_area, button)

Text(value='', description='API Key:', placeholder='Nhập API Key')

Text(value='', description='Domain:', placeholder='VD: dienmayxanh.com')

Dropdown(description='Quốc gia:', options=('vn', 'us', 'jp', 'au'), value='vn')

Dropdown(description='Ngôn ngữ:', options=('vi', 'en', 'ja', 'au'), value='vi')

Textarea(value='', description='Từ khóa:', layout=Layout(height='200px', width='500px'), placeholder='Nhập từ …

Button(description='Chạy', style=ButtonStyle())

Bấm **[API SERP](https://serper.dev)** để reg <b>API Free</b> nhé. Bấm **[Tạo Mail Ảo](https://generator.email)** để reg <b>mail ảo Free</b> nhé.<br><br>
Trong quá trình sử dụng tool nếu có lỗi xin hãy báo lại với mình. Để mình fix lỗi tool nhé các bạn.<Br>
**[Zalo](https://zalo.me/0966.523.518)** | **[Facebook](https://fb.com/buiduchanh2402)** | **[Website](https://buiduchanh.net)**

#[![](https://img.shields.io/badge/Tool%20làm%20bởi-Buiduchanh99tb-red)](https://www.facebook.com.vn/buiduchanh2402)  [![](https://img.shields.io/badge/Zalo%20-Kết%20nối-blue)](https://zalo.me/0966523518)

